In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from prophet import Prophet

In [ ]:
dados = yf.download("JNJ",start='2020-01-01',end='2023-12-31',progress=False)
dados = dados.reset_index()

In [ ]:
# Vamos dividir a nossa Base de Dados (treino -> ate o final do 1ª semestre de 2023) e etste (segundo semenstre de 2023)
dados_treino = dados[dados['Date'] < '2023-07-31']
dados_teste = dados[dados['Date'] >= '2023-07-31']

In [ ]:
# Preparando os Dados para o Prophet
dados_prophet_treino = dados_treino[['Date','Close']].rename(columns={'Date': 'ds','Close':'y'})

In [ ]:
dados_prophet_treino

In [ ]:
# Criar e Treinar o modelo.
modelo = Prophet(weekly_seasonality=True,yearly_seasonality=True,daily_seasonality=False)
modelo.add_country_holidays(country_name='US')
modelo.fit(dados_prophet_treino)

In [ ]:
# Criar datas futuras para previsao ate o final do ano de 2023
futuro = modelo.make_future_dataframe(periods=150)
previsao = modelo.predict(futuro)

In [ ]:
previsao.head(20)

In [ ]:
 # Plotar os Dados de Teino e Teste e previsao.
plt.figure(figsize=(14,8))
plt.plot(dados_treino['Date'],dados_treino['Close'],label='Dados de Treino',color='blue')
plt.plot(dados_teste['Date'],dados_teste['Close'],label='Dados Reais (Teste)',color='green')
plt.plot(previsao['ds'],previsao['yhat'],label='Previsao',color='orange',linestyle='--')

plt.axvline(dados_treino['Date'].max(),color='red',linestyle='--',label='Inicio da Previsao')
plt.xlabel('Date')
plt.ylabel('Preço de Fechamento')
plt.title('Previsao de Preço de Fechamento vs Dados Reias')
plt.legend()
plt.show()